In [7]:
import pandas as pd
import numpy as np

lan_df = pd.read_csv("/Users/renegonzalez/Desktop/DNN LoL/lan_dataset.csv")
na_df = pd.read_csv("/Users/renegonzalez/Desktop/DNN LoL/na_dataset.csv")
lan_df.head()


,BlueMastery1,BlueMastery2,BlueMastery3,BlueMastery4,BlueMastery5,BlueMasteryAvg,BlueMasteryMedian,BlueMasteryKurtorsis,BlueMasterySkewness,BlueMasteryStd,...,RedWinrates3,RedWinrates4,RedWinrates5,RedWinratesAvg,RedWinratesMedian,RedWinratesKurtorsis,RedWinratesSkewness,RedWinratesStd,RedWinratesVariance,DidBlueWon
0,302361,32548,137831,42344,2594552,621927.2,137831.0,4.804286,2.183943,991060.524461,...,0.485714,0.333333,0.588384,0.511934,0.552239,1.755027,-1.441162,0.097786,0.009562,0
1,244724,55894,166393,151398,17928,127267.4,151398.0,-1.336965,0.013165,81189.183929,...,0.551136,0.641509,0.500000,0.589079,0.551136,-0.950929,0.766032,0.077926,0.006073,0
2,1370461,165699,328554,11922,64623,388251.8,165699.0,4.066366,1.991324,502829.634615,...,0.631579,0.535714,0.529412,0.580125,0.535714,-0.171439,0.832254,0.092372,0.008533,0
3,859153,8207,152833,30736,94462,229078.2,94462.0,4.494974,2.098720,319077.677286,...,0.142857,0.515789,0.523632,0.336456,0.500000,-2.252575,-0.789161,0.221190,0.048925,1
4,295938,326564,1812645,64742,83541,516686.0,295938.0,4.480466,2.090117,656696.848512,...,0.565284,0.600000,0.416667,0.547907,0.565284,-1.091492,0.001576,0.094206,0.008875,1


In [8]:
frames = [lan_df, na_df]

result = pd.concat(frames)

# Convert the entire DataFrame
dataset = result.to_numpy()

np.random.shuffle(dataset)

X = dataset[:, 0:44]
Y = dataset[:, 44]

X.shape, Y.shape


((17010, 44), (17010,))

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators=55, learning_rate=0.14)


In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import sem

accuracy = []

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, random_state=None)

for train_index, test_index in skf.split(X, Y):
    x_train_fold, x_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = Y[train_index], Y[test_index]

    model.fit(x_train_fold, y_train_fold)
    prediction = model.predict(x_test_fold)
    score = accuracy_score(prediction, y_test_fold)
    accuracy.append(score * 100)


# Print the output.
print("List of possible accuracy: {}".format(accuracy))
print("\nMaximum Accuracy: {:.2f}%".format(max(accuracy)))
print("\nMinimum Accuracy: {:.2f}%".format(min(accuracy)))
print("\nOverall Accuracy: {:.2f}%".format(np.mean(accuracy)))
print("\nStandard Deviation is: {:.2f}%".format(np.std(accuracy)))
print("\nStandard Error is: {:.2f}%".format(sem(accuracy)))


List of possible accuracy: [89.94708994708994, 89.7119341563786, 89.2416225749559, 89.35920047031158, 89.12404467960023, 89.35920047031158, 89.0652557319224, 90.12345679012346, 89.41798941798942, 89.2416225749559]

Maximum Accuracy that can be obtained from this model is: 90.12%

Minimum Accuracy: 89.07%

Overall Accuracy: 89.46%

Standard Deviation is: 0.34%

Standard Error is: 0.11%


In [11]:
from sklearn.metrics import confusion_matrix, classification_report

model_final = GradientBoostingClassifier(n_estimators=55, learning_rate=0.14)
model_final.fit(X, Y)

print("Accuracy: {:.2f}%".format(model_final.score(X, Y) * 100))

predictions = model_final.predict(X)

print("Confusion Matrix:")
print(confusion_matrix(Y, predictions))

print("Classification Report:")
print(classification_report(Y, predictions))


Accuracy: 90.15%
Confusion Matrix:
[[7550  831]
 [ 845 7784]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.90      0.90      8381
         1.0       0.90      0.90      0.90      8629

    accuracy                           0.90     17010
   macro avg       0.90      0.90      0.90     17010
weighted avg       0.90      0.90      0.90     17010



In [12]:
import joblib

# save the model to disk
filename = "finalized_model.sav"
joblib.dump(model_final, filename)


['finalized_model.sav']